# Generate summary


Er zijn diverse algoritmes beschikbaar voor het maken van een tekst samenvatting. Gekozen is voor een op extractie gebaseerd algoritme, omdat deze techniek gebruikt maakt van bestaande zinnen uit de tekst en dit makkelijker is dan de abstractie technieken die nieuwe zinnen maakt aan de hand van NLP-technieken. Omdat er weinig voorbeeld samenvattingen beschikbaar waren, is er gekeken naar een extractie techniek die geen voorbeelden nodig heeft om van te leren. De gekozen extractie techniek gebruikt TextRank[1], een op graven gebaseerd NLP-rangschikkingsalgoritme. Zinnen die veel connecties hebben met andere zinnen zijn waarschijnlijk ‘centrale’ zinnen en worden hoger gescoord. Ook zinnen die veel Het samenvat-algoritme is als volgt:

1.	Een selectie uit de excel wordt omgezet naar een tekstfile
2.	De tekstfile wordt ingelezen en gesplitst op zinnen.
3.	Stopwoorden en rare tekens worden verwijderd. 
4.	Elke zin wordt omgezet in een vector (word embedding).
5.	Een matrix van m x m zinnen wordt gecreëerd waarin de overeenkomst wordt berekend tussen de zin-vectoren met behulp van cosine similarity
6.	De matrix wordt omgezet naar een graven netwerk waarin de zinnen de nodes zijn en de overeenkomst score een connectie.
7.	De rangvolgorde wordt berekend aan de hand van de scores en de plek in het netwerk.
8.	De N-hoogste zinnen worden geselecteerd voor de samenvatting.

Nota bene: Cosine similarity gaat er van uit dat alle woorden even belangrijk zijn. Een ander veel gebruikte overeenkomst score berekening is die van TFDIDF die ervan uitgaat dat sommige woorden belangrijker zijn dan andere woorden [3]. De euclidian distance berekening is minder geschikt voor dit soort tekst vector berekeningen.

Referenties

[1] Mihalcea, R. and Tarau, P., 2004, July. Textrank: Bringing order into text. In Proceedings of the 2004 conference on empirical methods in natural language processing (pp. 404-411).

[2] Allahyari, M., Pouriyeh, S., Assefi, M., Safaei, S., Trippe, E.D., Gutierrez, J.B. and Kochut, K., 2017. Text summarization techniques: a brief survey. arXiv preprint arXiv:1707.02268.

[3] Mallick, C., Das, A.K., Dutta, M., Das, A.K. and Sarkar, A., 2019. Graph-based text summarization using modified TextRank. In Soft computing in data analytics (pp. 137-146). Springer, Singapore.

[4] https://github.com/edubey/text-summarizer


In [1]:
#import libraries
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import pandas as pd


In [33]:
import nltk
import re
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

def get_clean_sentences(file_name):
    file = open(file_name, "r")
    text = file.read()
    text = text.replace(r"/\s/g", "")
    text = re.sub(r"(?<!\b[A-Z])\.(?!\d)", ". ", text) # this changes text.text into text. text
    sent_text = nltk.sent_tokenize(text) # this gives us a list of sentences
    #filter out sentences that have only .
    sent_text = [sent for sent in sent_text if len(sent) > 2]
    sent_text = [sent.capitalize() for sent in sent_text]
    # now loop over each sentence and tokenize it separately
    sentences = [tokenizer.tokenize(sentence) for sentence in sent_text]
    return sentences
    


In [34]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [42]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1][:10], sentences[idx2][:10], stop_words)

    return similarity_matrix


def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('dutch')
    summarize_text = []

    print('Step 1 - Read text and split it')
    sentences =  get_clean_sentences(file_name)

    print('Step 2 - Generate Similary Matrix across sentences')
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    print('Step 3 - Rank sentences in similarity matrix')
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    print('Step 4 - Sort the rank and pick top sentences')
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)     
    # fetch the top-n sentences (but only if they contain content)
    i = 0
    j = 0
    while (i < top_n):
        if len(ranked_sentence[j][1]) > 1:
            summarize_text.append(" ".join(ranked_sentence[j][1])+".")
            i = i+1
        j = j+1


    print('Step 5 - output the summarize text')
    print(f"\nSamenvatting {file_name}:")
    print("\n".join(summarize_text))


In [ ]:
# vul de lijst aan met documenten die geprocessed moeten worden
docs = ['toon.txt']
for file in docs:
    generate_summary(file, 10)      
    #try:
    #    generate_summary(file, 10)        
    #except:
    #    print('deze file kan niet geprocessed worden')

Step 1 - Read text and split it
Step 2 - Generate Similary Matrix across sentences
